In [18]:
# Initial imports
import pandas as pd 
import yfinance as yf
from datetime import datetime, timedelta
import time 

In [19]:
# Take S&P500 tickers from Wikipedia 
payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
second_table = payload[1]

sp500 = first_table

sp500.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [20]:
# Separate tickers 
tickers = sp500['Symbol'].tolist()

# Remove BRK.B and BF.B from tickers 
tickers.remove('BRK.B')
tickers.remove('BF.B')

In [21]:
# Set timeframe 
end_date = datetime.now()
start_date = (datetime.now() - timedelta(90))


In [22]:
final_OHLC_df = pd.DataFrame() # Declare the final empty dataframe

OHLC_data = pd.DataFrame() # Declare intermediate data frame

for ticker in tickers: # For each stock symbol in the list of symbols
      
    yf_ticker_obj = yf.Ticker(ticker) # Initiate object to get the data from API
        
    OHLC_data = yf_ticker_obj.history(start = start_date, end = end_date, interval = "1d", auto_adjust = True)
        
    OHLC_data = yf_ticker_obj.history(period= "ytd", interval = "1d", auto_adjust = True)
        
    OHLC_data.insert(0, 'Symbol', ticker) # Adding this data to dataframe
    
    OHLC_data = OHLC_data.drop(['Dividends','Stock Splits'], axis=1, errors='ignore')
        
    # Apppending this data to final dataframe
    final_OHLC_df = final_OHLC_df.append(OHLC_data)
    

In [23]:
# Delete split and dividend columns as this is not required.
        
OHLC_data = OHLC_data.drop(['Dividends','Stock Splits'], axis=1, errors='ignore')
        
# Apppending this data to final dataframe
final_OHLC_df = final_OHLC_df.append(OHLC_data)
    
final_OHLC_df.reset_index(inplace=True) # Re-setting the index
    
# Setting index to symbol and date
final_OHLC_df.set_index(["Date", "Symbol"], inplace=True)
    
  

In [34]:
final_OHLC_df["Returns"] = (final_OHLC_df["Close"] - final_OHLC_df["Open"])/final_OHLC_df["Open"]


final_OHLC_df.to_csv('../Project_3/S&P_500_Data')